# Watson OpenScale Mortgage Default Lab - Model Feed

Part two of a hands-on lab for IBM Watson OpenScale, this notebook should be run in a Watson Studio project with Python 3.6 or greater. This notebook will use the Watson Machine Learning client to send scoring requests to the model. You can specify the number of requests sent in the cells below.

Replace the credentials below with those of your Watson Machine Learning instance with the mortgage default model deployed.

In [ ]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": os.getenv('RUNTIME_ENV_APSX_URL'),
   "version": "3.5"
}

In the cell below, ensure that the model and deployment names are the same used for creating the model, and set the minimum and maximum number of records you wish to send to the model for scoring.

In [ ]:
MODEL_NAME = 'Mortgage Default'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

MIN_PAYLOADS = 101
MAX_PAYLOADS = 125

Using the credentials from above, create a Watson Machine Learning API client.

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

Get the deployment and model IDs from the WML client.

In [ ]:
space_name = "Mortage Default Production"
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
        print('Setting deployment space...')
        wml_client.set.default_space(space_id)
        

In [ ]:
wml_client.repository.list_models()

In [ ]:
deployment_id = None
for deployment in wml_client.deployments.get_details()['resources']:
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        deployment_id = deployment['metadata']['id']
print(deployment_id)

In [ ]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

In [ ]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)

In [ ]:
scoring_payload = {wml_client.deployments.ScoringMetaNames.INPUT_DATA:
                   [
                       {
                           "fields": data['fields'][1:],
                           "values": [],
                           "meta": {
                               "fields": ["ID"],
                               "values": []
                           }
                       }
                    ]
                  }

In [ ]:
import random
import string

letters = string.digits

for _ in range(0, 101):
    value_to_score = random.choice(data['values'])
    scoring_payload['input_data'][0]['values'].append(value_to_score[1:])
    scoring_payload['input_data'][0]['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])
print(len(scoring_payload['input_data'][0]['values']))

In [ ]:
predictions = wml_client.deployments.score(deployment_id, scoring_payload)

In [ ]:
predictions